# Prepare data for GELATO run
Load crossmatch results and monitor them. Prepare data for emission/absorption lines identification with GELATO.

## 1. Ensure path is known to the system

In [ ]:
import os

os.environ["FORS2DATALOC"]

In [ ]:
if os.environ["FORS2DATALOC"] == "":
    os.environ["FORS2DATALOC"] = os.path.abspath(os.path.join("..", "..", "src", "data"))
os.environ["FORS2DATALOC"]

It is strongly recommended to add the following to your `.bashrc` or `.bash_aliases` file:
```bash
export FORS2DATALOC="[path to this repository]/src/data"
```
Then log out and log back in, or `source` the file, and the environment variable will be set and should be set automatically each time you start a session.

## 2. Load and glance at crossmatch data

In [ ]:
from process_fors2.analysis import loadDataInH5

In [ ]:
filename = "resulting_merge_from_walkthrough.h5"
xmatchfile = os.path.abspath(os.path.join(".", filename))
if os.path.isfile(xmatchfile):
    print(f"{xmatchfile} is a valid HDF5 file.")

The function `loadDataInH5` extracts a dictionary of data from the HDF5 file, for a single galaxy. Let's pick one and go through the process.

In [ ]:
import h5py
import re
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
with h5py.File(xmatchfile, "r") as xfile:
    tags = np.array(list(xfile.keys()))
    print(tags[:10])
    tag0 = tags[0]
    group0 = xfile.get(tag0)
    print(group0.attrs.get("name"), group0.attrs.get("num"), tag0)
    nums = []
    for tag in tags:
        group = xfile.get(tag)
        num = group.attrs.get("num")
        nums.append(num)
nums = np.array(nums)
nums

In [ ]:
specid = np.random.choice(nums)
specid

In [ ]:
datadict = loadDataInH5(specid, h5file=xmatchfile)

In [ ]:
datadict

In [ ]:
from sedpy import observate

In [ ]:
filts = observate.load_filters(["sdss_u0", "sdss_g0", "sdss_r0", "sdss_i0"])
mags = [datadict["MAG_GAAP_u"], datadict["MAG_GAAP_g"], datadict["MAG_GAAP_r"], datadict["MAG_GAAP_i"]]
magserr = [datadict["MAGERR_GAAP_u"], datadict["MAGERR_GAAP_g"], datadict["MAGERR_GAAP_r"], datadict["MAGERR_GAAP_i"]]

In [ ]:
sdss_u, sdss_g, sdss_r, sdss_i = filts

In [ ]:
f, a = plt.subplots(1, 1)
aa = a.twinx()
for filt, c in zip(filts, ["b", "g", "y", "r"]):
    aa.fill_between(filt.wavelength, filt.transmission, color=c, alpha=0.4, label=filt.name)
    aa.axvline(filt.blue_edge, lw=0.5, c=c, ls=":")
    aa.axvline(filt.red_edge, lw=0.5, c=c, ls=":")
    aa.axvline(filt.wave_effective, lw=0.5, c=c, ls="-")
sel = np.where(datadict["mask_f2"] > 0, False, True)
a.plot(datadict["wl_f2"][sel], datadict["fl_f2"][sel], lw=0.5, c="k", label=datadict["name"])
a.set_xlabel("Wavelength $[ \AA ]$")
a.set_ylabel("Spectral flux [arbitrary units]")
aa.set_ylabel("Filter transmission")
f.legend(loc="lower left", bbox_to_anchor=(1.01, 0.0))

## 3. Scale spectral flux on photometry
We start by identifying bands in which we can compute a flux from the spectrum, then a factor to apply on this spectrum tu match photometry from KiDS in these bands.

In [ ]:
good_filts = []
good_mags = []
good_magserr = []
for f, m, err in zip(filts, mags, magserr):
    if (f.blue_edge > min(datadict["wl_f2"][sel])) and (f.red_edge < max(datadict["wl_f2"][sel])) and np.isfinite(m) and np.isfinite(err):
        good_filts.append(f)
        good_mags.append(m)
        good_magserr.append(err)

In [ ]:
print(good_filts, good_mags, good_magserr)

In [ ]:
from process_fors2.analysis import scalingToBand

In [ ]:
flux2phot = scalingToBand(datadict["wl_f2"], datadict["fl_f2"], good_mags, good_magserr, mask=datadict["mask_f2"], band=[f.name for f in good_filts])

In [ ]:
f, a = plt.subplots(1, 1)
aa = a.twinx()
for filt in good_filts:
    aa.fill_between(filt.wavelength, filt.transmission, alpha=0.4, label=filt.name)
sel = np.where(datadict["mask_f2"] > 0, False, True)
a.plot(datadict["wl_f2"][sel], flux2phot * datadict["fl_f2"][sel], lw=0.5, c="k", label=datadict["name"])
a.set_xlabel("Wavelength $[ \AA ]$")
a.set_ylabel("Spectral flux $[\mathrm{erg} . \mathrm{s}^{-1} . \mathrm{cm}^{-2} . \mathrm{\AA}^{-1}]$")
aa.set_ylabel("Filter transmission")
f.legend(loc="lower left", bbox_to_anchor=(1.01, 0.0))

In [ ]:
f, a = plt.subplots(1, 1)
aa = a.twinx()
sel = np.where(datadict["mask_f2"] > 0, False, True)
for idx, (filt, m, err) in enumerate(zip(good_filts, good_mags, good_magserr)):
    aa.fill_between(filt.wavelength, filt.transmission, alpha=0.4, label=filt.name)
    mab = filt.ab_mag(datadict["wl_f2"][sel], flux2phot * datadict["fl_f2"][sel])
    lab = "Magnitude from scaling"
    if idx > 0:
        lab = ""
    a.scatter(filt.wave_effective, mab, s=49, marker="x", label=lab)
    lab = "Magnitude from observation"
    if idx > 0:
        lab = ""
    a.errorbar(filt.wave_effective, m, err, fmt="s", label=lab)
a.set_xlabel("Wavelength $[ \AA ]$")
a.set_ylabel("Magnitude $[\mathrm{AB}]$")
aa.set_ylabel("Filter transmission")
f.legend(loc="lower left", bbox_to_anchor=(1.01, 0.0))

We can see that the spectrum has been rescaled so that magnitudes in selected bands match as closely as possible.

## 4. Estimate signal and noise in FORS2 spectra
In order for GELATO to run, we must provide it with an estimation of errors in the spectra. Therefore, we perform a rough estimation of signal and noise in the data, base on gaussian filtering.

In [ ]:
from process_fors2.analysis import estimateErrors

In [ ]:
scaled_flux = flux2phot * datadict["fl_f2"]
fl_signal, fl_noise = estimateErrors(datadict["wl_f2"], scaled_flux, mask=datadict["mask_f2"], nsigma=3)

## 5. Write results for GELATO
GELATO reads `FITS` files containing the data.

In [ ]:
from astropy.io import fits
from astropy.table import Table

### Format data
From the GELATO [respository](https://github.com/TheSkyentist/GELATO) :

In order to run GELATO you need:
[...]
- The spectrum or spectra. The log10 of the wavelength in Angstroms of the spectrum must be provided along with the spectral flux density per unit wavelength (Flam). The inverse variance of the fluxes, in corresponding units, must also be provided.
- The redshift of each spectrum. The redshift of the object must be passed to construct the spectrum object. While the redshift is a fitted parameter, the provided value must be correct to at least 1 part in 200, preferable 1 part in 1000. A basic estimate from the apparent position of any identified emission line should suffice.

In [ ]:
nomask = np.where(datadict["mask_f2"] > 0, False, True)
sel = np.logical_and(nomask, datadict["fl_f2"] > 0)
sel = np.logical_and(sel, np.isfinite(fl_noise))
sel = np.logical_and(sel, fl_noise > 0)
wl_gel = np.log10(datadict["wl_f2"][sel])
flam_gel = flux2phot * datadict["fl_f2"][sel]
inv_var = np.power(fl_noise[sel], -2)

redz = datadict["redshift"]

plt.plot(wl_gel, flam_gel)
print(f"redshift: {redz:.3f}")

### Write data
Save data in appropriate files for later use in GELATO.

Gathering Ingredients: First, the spectrum is loaded. The code assumes the spectrum file is a FITS table with the following columns and column names:

1. The log10 of the wavelengths in Angstroms, column name: "loglam"
2. The spectral flux density in flam units, column name: "flux"
3. The inverse variances of the data points, column name: "ivar"

In [ ]:
t = Table([wl_gel, flam_gel, inv_var], names=["loglam", "flux", "ivar"])

In [ ]:
t

In [ ]:
fpath = os.path.abspath(f"{datadict['name']}_z{redz:.3f}_GEL.fits")
if False:
    t.write(fpath, format="fits", overwrite=True)

The data has been written in a `FITS` file built to work with GELATO.

In [ ]:
objlist = Table([[fpath], [redz]], names=["Path", "z"])

In [ ]:
objlist

In [ ]:
if False:
    objlist.write("specs_for_GELATO.fits", format="fits", overwrite=True)

The list of data files has been written in a `FITS` file built to work with GELATO.

This is the end of this notebook. All steps will be available in a dedicated function so that this process can be looped and all FORS2 spectra processed in an identical way for future, well-controlled GEALTO runs.

## 6. Loop

In [ ]:
from process_fors2.analysis import crossmatchToGelato

In [ ]:
outdir = "./test_gelato"

In [ ]:
tab, path = crossmatchToGelato(filename, outdir)

In [ ]:
tab

And *voilà !*, all data files for GELATO have been generated correctly... hopefully.